In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
dataset = "C:/Users/g/OneDrive/Desktop/AIML/project1/House_Rent_Dataset.csv"
#df = pd.read_csv(io.StringIO(dataset) , sep=r'\s+')  # Replace with actual dataset path
df = pd.read_csv(dataset)
df

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner
...,...,...,...,...,...,...,...,...,...,...,...,...
4741,2022-05-18,2,15000,1000,3 out of 5,Carpet Area,Bandam Kommu,Hyderabad,Semi-Furnished,Bachelors/Family,2,Contact Owner
4742,2022-05-15,3,29000,2000,1 out of 4,Super Area,"Manikonda, Hyderabad",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Owner
4743,2022-07-10,3,35000,1750,3 out of 5,Carpet Area,"Himayath Nagar, NH 7",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Agent
4744,2022-07-06,3,45000,1500,23 out of 34,Carpet Area,Gachibowli,Hyderabad,Semi-Furnished,Family,2,Contact Agent


In [3]:
def adjust_size(row):
    if row['Area Type'] == 'Super Area':
        return row['Size'] * 0.75
    elif row['Area Type'] == 'Carpet Area':
        return row['Size'] * 1.3
    else:
        return row['Size']
        
df['final_size'] = df.apply(adjust_size, axis=1)

In [4]:
out1 = df[df['final_size'] > 6000].index
out1
df.drop(out1, inplace=True)
ou2 = df[df['Rent'] > 300000].index
ou2
df.drop(ou2, inplace=True)

In [5]:
rep = { 'Ground': 0, 'Lower': 0,'Upper' : 0}
dffloor = df['Floor'].str.split().str[0].replace(rep)
dffloor
df.insert(3,'FL', dffloor)

In [6]:
df.head(5)

,Posted On,BHK,Rent,FL,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,final_size
0,2022-05-18,2,10000,0,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner,825.0
1,2022-05-13,2,20000,1,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,600.0
2,2022-05-16,2,17000,1,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,750.0
3,2022-07-04,2,10000,1,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner,600.0
4,2022-05-09,2,7500,1,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner,1105.0


In [8]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
columns_to_encode = ['City', 'Furnishing Status', 'Tenant Preferred','Point of Contact']

In [17]:
encoded_data = encoder.fit_transform(df[columns_to_encode])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_encode), index=df.index)

In [16]:
df

,Posted On,BHK,Rent,FL,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,final_size
0,2022-05-18,2,10000,0,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner,825.0
1,2022-05-13,2,20000,1,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,600.0
2,2022-05-16,2,17000,1,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner,750.0
3,2022-07-04,2,10000,1,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner,600.0
4,2022-05-09,2,7500,1,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner,1105.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4741,2022-05-18,2,15000,3,1000,3 out of 5,Carpet Area,Bandam Kommu,Hyderabad,Semi-Furnished,Bachelors/Family,2,Contact Owner,1300.0
4742,2022-05-15,3,29000,1,2000,1 out of 4,Super Area,"Manikonda, Hyderabad",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Owner,1500.0
4743,2022-07-10,3,35000,3,1750,3 out of 5,Carpet Area,"Himayath Nagar, NH 7",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Agent,2275.0
4744,2022-07-06,3,45000,23,1500,23 out of 34,Carpet Area,Gachibowli,Hyderabad,Semi-Furnished,Family,2,Contact Agent,1950.0


In [13]:
other_cols = df[['BHK', 'FL', 'Bathroom', 'final_size']]

In [18]:
final_df = pd.concat([other_cols, encoded_df], axis=1)
final_df

,BHK,FL,Bathroom,final_size,City_Bangalore,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Furnishing Status_Furnished,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Tenant Preferred_Bachelors,Tenant Preferred_Bachelors/Family,Tenant Preferred_Family,Point of Contact_Contact Agent,Point of Contact_Contact Builder,Point of Contact_Contact Owner
0,2,0,2,825.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,1,1,600.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,2,1,1,750.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,2,1,1,600.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4,2,1,1,1105.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4741,2,3,2,1300.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4742,3,1,3,1500.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4743,3,3,3,2275.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4744,3,23,2,1950.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [19]:
X = final_df.values  # or X.to_numpy()
y = df['Rent'].values

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # Fit and transform on TRAINING data
X_test = scaler.transform(X_test)  

In [38]:
tf.random.set_seed(1234)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(19, activation='relu', input_shape=(X_train.shape[1],), name = "L1"), # Input layer
    tf.keras.layers.Dense(8, activation='relu', name = "L2"), # Hidden layer
    tf.keras.layers.Dense(1) # Output layer (for regression - adjust if classification)
])

###19,8,1 ---3 layer --- 1000 epochs r2 score 0.768747091293335
####19,12,8,1 ----4 layers --- 2000 epochs R-squared Score: 0.7712708115577698

In [41]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [40]:
model.fit(X_train, y_train, epochs=1000)

Epoch 1/1000
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2703843328.0000 - mae: 30842.5762    
Epoch 2/1000
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2703538176.0000 - mae: 30839.1562 
Epoch 3/1000
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2701937920.0000 - mae: 30825.8867 
Epoch 4/1000
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2696907008.0000 - mae: 30789.4902 
Epoch 5/1000
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2686214912.0000 - mae: 30715.9766 
Epoch 6/1000
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2668023040.0000 - mae: 30592.8008 
Epoch 7/1000
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2640218368.0000 - mae: 30405.6094 
Epoch 8/1000
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2601489920.0000 - mae: 30145.8633 
Epoch 9/1000
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2551765760.0000 - mae: 29810.6699 
Epoch 10/1000
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2490998528.0000 - mae: 29396.7656 
Epoch 11/1000
118/118 ━━━━

In [42]:
y_pred = model.predict(X_test)

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [43]:
r2 = r2_score(y_test, y_pred)

print(f"R-squared Score: {r2}")

R-squared Score: 0.7665455341339111
